In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint,
    MeshcatContactVisualizer
)

import pickle

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/ground.urdf"), "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_test.urdf"), "cracker_box")


In [4]:
plant.Finalize()
plant.set_name("plant")

In [5]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
# meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
#                  axis_radius=0.001,)

# #visualizer the contact
# contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
#                                        force_threshold=0.0,
#                                        contact_force_scale=0.05, 
#                                        contact_force_radius=0.001,   
#                                        plant=plant))
# builder.Connect(plant.get_contact_results_output_port(),
#                 contact_visualizer.GetInputPort("contact_results"))


diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
X_WB = RigidTransform(RotationMatrix(), [0., 0., 0.0334 + 1e-07])
plant.SetFreeBodyPose(plant_context, 
                              plant.GetBodyByName("base_link_cracker", cracker_box),
                              X_WB)

# meshcat.load()
diagram.Publish(context)

In [6]:
# q0_o = plant.GetPositions(plant_context, cracker_box)
p = plant.CalcPointsPositions(plant_context, plant.GetFrameByName('point_collision5'), [0,0,0], plant.world_frame())[2]
print(p)
# print(diagram)
# ad_diagram = diagram.ToAutoDiffXd()
# ad_plant = ad_diagram.GetSubsystemByName("plant")
# ad_context = ad_diagram.CreateDefaultContext()
# ad_plant_context = ad_plant.GetMyContextFromRoot(ad_context)

[1.e-07]


In [12]:
# 2-D array: 2 x 3
# two_dim_matrix_one = np.array([[1, 2, 3], [4, 5, 6]])
# another_two_dim_matrix_one = np.array([[7, 8, 9], [4, 7, 1]]).reshape(3, 2)

# two_dim_matrix_one = [[1, 2, 3], [4, 5, 6]]
# another_two_dim_matrix_one = [[7, 8, 9], [4, 7, 1]]
another_two_dim_matrix_one = np.array([[7, 8],[9 ,4], [7, 1]])

two_dim_matrix_one = np.array([1, 2, 3])
print('element wise product: %s' %(another_two_dim_matrix_one.T))
print('element wise product: %s' %(another_two_dim_matrix_one.T.dot(two_dim_matrix_one)))
# 对应元素相乘 element-wise product
# element_wise = two_dim_matrix_one * another_two_dim_matrix_one
# print('element wise product: %s' %(element_wise))
 
# 对应元素相乘 element-wise product
# element_wise_2 = np.dot(two_dim_matrix_one, another_two_dim_matrix_one)
# print('element wise product: %s' % (element_wise_2))

# element_wise_2 = two_dim_matrix_one.dot(another_two_dim_matrix_one)
# print('element wise product: %s' % (element_wise_2))

# another_two_dim_matrix_one = np.array([7, 8, 9])
# two_dim_matrix_one = np.array([1, 2, 3]).reshape(3, 1)
# print('element wise product: %s' %(another_two_dim_matrix_one.T))
# tt = np.zeros((nv_o))
# tt = tt + tt
# print('tt: %s' %(tt.reshape(6,1)))

element wise product: [[7 9 7]
 [8 4 1]]
element wise product: [46 19]


In [6]:
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
scene_graph_inspector = query_object.inspector()
collision_candidate_pairs = scene_graph_inspector.GetCollisionCandidates()

cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())
ground_body = plant.GetBodyByName("ground", ground)
ground_id = plant.GetBodyFrameIdIfExists(ground_body.index())
cracker_box_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::box_collision")
ground_geometry_id = scene_graph_inspector.GetGeometryIdByName(ground_id, pydrake.geometry.Role.kProximity, "ground::ground_collision")
ignored_collision_pairs = set([(ground_geometry_id, cracker_box_geometry_id)])
# print("ignored_collision_pairs: ", ignored_collision_pairs)
# if (ground_geometry_id, cracker_box_geometry_id) in ignored_collision_pairs:
#     print("ground_id: ", ground_id, "   cracker_box_id: ", cracker_box_id)

# collision_candidate_pair_and_f_list = []
# i = 0
# for n in range(1):
#     collision_candidate_pair_and_f = {}
#     for collision_candidate_pair in collision_candidate_pairs:
#         print("id_A: ", collision_candidate_pair[0], "   id_B: ", collision_candidate_pair[1])
#         print("collision_candidate_pair: ", collision_candidate_pair)
#         collision_candidate_pair_and_f[collision_candidate_pair] = i
#         i = i + 1
#     collision_candidate_pair_and_f_list.append(collision_candidate_pair_and_f)
#     print("+++++++++++++++++++++++")
# for collision_candidate_pair_and_f in collision_candidate_pair_and_f_list:
#     for key, f in collision_candidate_pair_and_f.items():

#         print("id_A: ", key[0], "   id_B: ", key[1])
#         print("f: ", f)
# print("------------------------")
# signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()
# for signed_distance_pair in signed_distance_pairs:
#     print("id_A: ", signed_distance_pair.id_A, "   id_B: ", signed_distance_pair.id_B)
#     print("p_ACa: ", signed_distance_pair.p_ACa, "   p_BCb: ", signed_distance_pair.p_BCb)
#     print("nhat_BA_W: ", signed_distance_pair.nhat_BA_W, "   distance: ", signed_distance_pair.distance)
#     print("########################")
    
    
    
 #f is A exert to B express in W
for collision_candidate_pair in collision_candidate_pairs:
#     if collision_candidate_pair in ignored_collision_pairs:
#         print("collision_candidate_pair: ", collision_candidate_pair)
    
    signed_distance_pair = query_object.ComputeSignedDistancePairClosestPoints(
                                collision_candidate_pair[0], collision_candidate_pair[1])
    print("id_A: ", signed_distance_pair.id_A, "   id_B: ", signed_distance_pair.id_B)
    print("distance: ", signed_distance_pair.distance)
    print("########################")
    
    
#     frame_A_id = scene_graph_inspector.GetFrameId(signed_distance_pair.id_A)
#     frame_B_id = scene_graph_inspector.GetFrameId(signed_distance_pair.id_B)
#     frameA = plant.GetBodyFromFrameId(frame_A_id).body_frame()
#     frameB = plant.GetBodyFromFrameId(frame_B_id).body_frame()
#     print("frameA: ", frameA, "   frameB: ", frameB)
#     print("frame_A_id: ", frame_A_id, "   frame_B_id: ", frame_B_id)
    
#     geometry_A_instance = scene_graph_inspector.CloneGeometryInstance(signed_distance_pair.id_A)
#     geometry_B_instance = scene_graph_inspector.CloneGeometryInstance(signed_distance_pair.id_B)
#     print("geometry_A_instance: ", geometry_A_instance.name(), "   geometry_B_instance: ", geometry_B_instance.name())

id_A:  <GeometryId value=4>    id_B:  <GeometryId value=10>
distance:  0.0668
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=8>
distance:  0.0005001000000000033
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=18>
distance:  2.875570976809943e-18
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=14>
distance:  0.0668
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=16>
distance:  0.0668
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=12>
distance:  0.0668
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=22>
distance:  2.875570976809943e-18
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=24>
distance:  2.875570976809943e-18
########################
id_A:  <GeometryId value=4>    id_B:  <GeometryId value=20>
distance:  2.875570976809943e-18
########################

In [6]:
init_from_file = False
# init_from_file = True
tmpfolder = 'tmp/'
with open(tmpfolder +  'test_only_box_passive_scenegraph/sol.pkl', 'rb' ) as file:
          q_o_ini, v_o_ini, vdot_o_ini, f_list_ini, alpha_beta_list_ini = pickle.load( file )

In [7]:
# Need this because a==b returns True even if a = AutoDiffXd(1, [1, 2]), b= AutoDiffXd(2, [3, 4])
# That's the behavior of AutoDiffXd in C++, also.
def autoDiffArrayEqual(a,b):
    return np.array_equal(a, b) and np.array_equal(autoDiffToGradientMatrix(a), autoDiffToGradientMatrix(b))


#set contact sequence
N = 100
# in_contact = np.zeros((N))
# in_contact[40:] = 1


# Create one context per timestep (to maximize cache hits)
#to use scenegraph, so the plant context need derive from diagram
context_list = [diagram.CreateDefaultContext() for i in range(N)]
plant_context_list = [plant.GetMyContextFromRoot(context_list[i]) for i in range(N)]
# We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
ad_diagram = diagram.ToAutoDiffXd()
ad_plant = ad_diagram.GetSubsystemByName("plant")

# Make a new autodiff context (to maximize cache hits)
#to use scenegraph, so the plant context need derive from diagram
ad_context_list = [ad_diagram.CreateDefaultContext() for i in range(N)]
ad_plant_context_list = [ad_plant.GetMyContextFromRoot(ad_context_list[i]) for i in range(N)]




mu = 0.5 
total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(cracker_box))
gravity = plant.gravity_field().gravity_vector()



#time step constrain
T = 1
prog = MathematicalProgram()
# Time steps    
h = prog.NewContinuousVariables(N-1, "h")
prog.AddBoundingBoxConstraint(0.5*T/N, 2.0*T/N, h) 
prog.AddLinearConstraint(sum(h) >= .9*T)
prog.AddLinearConstraint(sum(h) <= 1.1*T)


#q, v, vdot constrain
nq_o = plant.num_positions(cracker_box)
nv_o = plant.num_velocities(cracker_box)
q_o = prog.NewContinuousVariables(nq_o, N, "q_o")
v_o = prog.NewContinuousVariables(nv_o, N, "v_o")
vdot_o = prog.NewContinuousVariables(nv_o, N, "vdot_o")

#initial constrain
q0_o = plant.GetPositions(plant_context, cracker_box)
v0_o = plant.GetVelocities(plant_context, cracker_box)
prog.AddBoundingBoxConstraint(q0_o, q0_o, q_o[:,0])
prog.AddBoundingBoxConstraint(v0_o, v0_o, v_o[:,0])
# prog.AddBoundingBoxConstraint([0]*nv_o, [0]*nv_o, vdot_o[:,0])
for n in range(N):
    if not init_from_file:
        prog.SetInitialGuess(q_o[:,n], q0_o)  
    # Unit quaternions
    AddUnitQuaternionConstraintOnPlant(plant, q_o[:,n], prog)
    # Running costs:
#     prog.AddQuadraticErrorCost(np.diag([1]*nq_o), q0_o, q_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([1]*nv_o), [0]*nv_o, v_o[:,n])
#     prog.AddQuadraticErrorCost(np.diag([10]*nv_o), [0]*nv_o, vdot_o[:,n])

    
if init_from_file:
    prog.SetInitialGuess(q_o, q_o_ini)
    prog.SetInitialGuess(v_o, v_o_ini)
    prog.SetInitialGuess(vdot_o, vdot_o_ini)

In [8]:
def velocity_position_constraint(vars, context_index):
    h, q, v, qn = np.split(vars, [1, 1+nq_o, 1+nq_o+nv_o])
    if isinstance(vars[0], AutoDiffXd):
        if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
            ad_plant.SetPositions(ad_plant_context_list[context_index], q)
        v_from_qdot = ad_plant.MapQDotToVelocity(ad_plant_context_list[context_index], (qn - q)/h)
    else:
        if not np.array_equal(q, plant.GetPositions(plant_context_list[context_index])):
            plant.SetPositions(plant_context_list[context_index], q)
        v_from_qdot = plant.MapQDotToVelocity(plant_context_list[context_index], (qn - q)/h)
    return v - v_from_qdot
for n in range(N-1):
    prog.AddConstraint(
        partial(velocity_position_constraint, context_index=n), 
        lb=[0]*nv_o, ub=[0]*nv_o, 
        vars=np.concatenate(([h[n]], q_o[:,n], v_o[:,n+1], q_o[:,n+1])))  #(implicit Euler)
    
#accelaration and velocity relation constrain(implicit Euler)
for n in range(N-1):
#     prog.AddConstraint(eq(q_o[:, n+1], q_o[:,n] + h[n]*v_o[:,n]))
    prog.AddConstraint(eq(v_o[:, n+1], v_o[:,n] + h[n]*vdot_o[:,n+1]))

In [9]:
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
scene_graph_inspector = query_object.inspector()
collision_candidate_pairs = scene_graph_inspector.GetCollisionCandidates()

#set the ignored_collision_pairs
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())
ground_body = plant.GetBodyByName("ground", ground)
ground_id = plant.GetBodyFrameIdIfExists(ground_body.index())
ground_geometry_id = scene_graph_inspector.GetGeometryIdByName(ground_id, pydrake.geometry.Role.kProximity, "ground::ground_collision")
box_collision_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::box_collision")
point_collision1_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision1")
point_collision2_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision2")
point_collision3_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision3")
point_collision4_geometry_id = scene_graph_inspector.GetGeometryIdByName(cracker_box_id, pydrake.geometry.Role.kProximity, "cracker_box::point_collision4")

ignored_collision_pairs = set([(ground_geometry_id, box_collision_geometry_id)
#                                ,(ground_geometry_id, point_collision1_geometry_id)
#                                ,(ground_geometry_id, point_collision2_geometry_id)
#                                ,(ground_geometry_id, point_collision3_geometry_id)
#                                ,(ground_geometry_id, point_collision4_geometry_id)
                              ])
collision_pairs1 = set([
                               (ground_geometry_id, point_collision1_geometry_id)
                               ,(ground_geometry_id, point_collision2_geometry_id)
                               ,(ground_geometry_id, point_collision3_geometry_id)
                               ,(ground_geometry_id, point_collision4_geometry_id)
                              ])



#declare the force and alpha beta decision variables, set collision_candidate_pair_list
collision_candidate_pair_list = []
f_list = []
alpha_beta_list = []
for n in range(N):
    f = []
    alpha_beta = []
    for collision_candidate_pair in collision_candidate_pairs:
        if collision_candidate_pair not in ignored_collision_pairs:
            f.append(prog.NewContinuousVariables(3, 1, f"collision_point_{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_contact_force"))
            alpha_beta.append(prog.NewContinuousVariables(2, 1, f"{collision_candidate_pair[0]}_{collision_candidate_pair[1]}_{n}_alpha_beta"))
            if n == 0 :
                collision_candidate_pair_list.append(collision_candidate_pair)
    alpha_beta_list.append(alpha_beta)
    f_list.append(f)

N_f = len(collision_candidate_pair_list)

#set initial guess of f and alpha beta
if init_from_file:
    for n in range(N):
        for k in range(N_f):
            prog.SetInitialGuess(f_list[n][k], f_list_ini[n][k])
            prog.SetInitialGuess(alpha_beta_list[n][k], alpha_beta_list_ini[n][k])
else:
    f0 = np.array([0, 0, (-gravity[2])*total_mass/4])
    alpha0 = (-gravity[2])*total_mass/4
    beta0 = 0
    alpha_beta0 = np.array([alpha0, beta0])
    for n in range(N):
        for k in range(N_f):
            if collision_candidate_pair_list[k] not in collision_pairs1:
                prog.SetInitialGuess(f_list[n][k], f0)
                prog.SetInitialGuess(alpha_beta_list[n][k], alpha_beta0)
        

        
        
        
# print(collision_candidate_pair_list)
# for n in range(N_f):
#     signed_distance_pair = query_object.ComputeSignedDistancePairClosestPoints(
#                                 collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
#     print("id_A: ", signed_distance_pair.id_A, "   id_B: ", signed_distance_pair.id_B)
#     print("distance: ", signed_distance_pair.distance)
#     print("########################")

In [10]:
def phiq_and_force_constrain(vars, context_index):
    q, v, alpha_beta, f = np.split(vars, [nq_o, nq_o+nv_o, nq_o+nv_o+N_f*2])
    #f and alpha_beta contain all collsion candidates force and corresponding alpha,beta
    #1 dimension vector reshape to needed dimension
    alpha_beta = np.array(alpha_beta).reshape(N_f, 2)
    f = np.array(f).reshape(N_f, 3)

    
    #set plant pos and vel
    if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
        ad_plant.SetPositions(ad_plant_context_list[context_index], q)
    if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
        ad_plant.SetVelocities(ad_plant_context_list[context_index], v)
    
    #get query object and scenegraph inspector
    ad_query_object = ad_plant.get_geometry_query_input_port().Eval(ad_plant_context_list[context_index])
    ad_scene_graph_inspector = ad_query_object.inspector()
    
    res_list = []
    #f is A exert to B express in W
    for n in range(N_f):
        signed_distance_pair = ad_query_object.ComputeSignedDistancePairClosestPoints(
                                    collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
        #derive friction coefficience
        geometryA_props = scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_A)
        geometryB_props = scene_graph_inspector.GetProximityProperties(signed_distance_pair.id_B)
        geometryA_friction = geometryA_props.GetProperty("material", "coulomb_friction")
        geometryB_friction = geometryB_props.GetProperty("material", "coulomb_friction")
        combined_friction = pydrake.multibody.plant.CalcContactFrictionFromSurfaceProperties(
                                                        geometryA_friction, geometryB_friction)
        
        #static friction cone complementarity constrain
        nhat_BA_W = np.array([signed_distance_pair.nhat_BA_W]).T
        
        res = np.concatenate((
            [alpha_beta[n][0]],
            [alpha_beta[n][1]],
            (np.array([f[n]]).dot(((combined_friction.static_friction()**2 + 1) * nhat_BA_W*nhat_BA_W.T - np.identity(3)).dot(np.array([f[n]]).T))).flatten(),
            (-nhat_BA_W.T.dot(f[n])).flatten() - [alpha_beta[n][0]],  
            [signed_distance_pair.distance - alpha_beta[n][1]], 
            [alpha_beta[n][0] * alpha_beta[n][1]])) 
#         print("complement:", [alpha_beta[n][0] * alpha_beta[n][1]], "      phiq:", [signed_distance_pair.distance]
#              ,"      fn:", (-nhat_BA_W.T.dot(f[n])).flatten())
        res_list.append(res)
    #return must be 1 dimension vector, like [r1, r2, ...]
    res_list = np.array(res_list).reshape(6*N_f,)
    return res_list


epsilon = 1e-10
for n in range(N):
#     print()
    #set vars be 1 dimension vector, like [r1, r2, ...]
    vars = np.concatenate((q_o[:, n], v_o[:, n], 
                           np.array(alpha_beta_list[n]).flatten(), 
                        np.array(f_list[n]).flatten()))
    prog.AddConstraint(
        partial(phiq_and_force_constrain, context_index=n), 
        lb=[0]*6*N_f, ub=np.array([np.concatenate(([4*(-gravity[2])*total_mass], [np.inf]*2, [0]*2, [epsilon])) for i in range(N_f)]).reshape(6*N_f,), 
        vars=vars)
#     prog.AddConstraint(
#         partial(phiq_and_force_constrain, context_index=n), 
#         lb=[0]*6*N_f, ub=np.array([np.concatenate(([4*(-gravity[2])*total_mass], [np.inf]*2, [np.inf]*2, [epsilon])) for i in range(N_f)]).reshape(6*N_f,), 
#         vars=vars) #not use alpha beta
#     prog.AddConstraint(
#         partial(phiq_and_force_constrain, context_index=n), 
#         lb=[0]*6*N_f, ub=np.array([np.concatenate(([4*(-gravity[2])*total_mass], [0], [np.inf], [0]*2, [epsilon])) for i in range(N_f)]).reshape(6*N_f,), 
#         vars=vars)  #phiq_is_zero

In [11]:
def manipulator_equations(vars, context_index):
    q, v, vdot, f = np.split(vars, [nq_o, nq_o+nv_o, nq_o+nv_o*2])
    #f and alpha_beta contain all collsion candidates force and corresponding alpha,beta
    f = np.array(f).reshape(N_f, 3)
    
    #set plant pos and vel
    if not autoDiffArrayEqual(q, ad_plant.GetPositions(ad_plant_context_list[context_index], cracker_box)):
        ad_plant.SetPositions(ad_plant_context_list[context_index], q)
    if not autoDiffArrayEqual(v, ad_plant.GetVelocities(ad_plant_context_list[context_index], cracker_box)):
        ad_plant.SetVelocities(ad_plant_context_list[context_index], v)
    
    # matrices for the manipulator equations
    M = ad_plant.CalcMassMatrixViaInverseDynamics(ad_plant_context_list[context_index])
    Cv = ad_plant.CalcBiasTerm(ad_plant_context_list[context_index])
    tauG = ad_plant.CalcGravityGeneralizedForces(ad_plant_context_list[context_index])
#     print("tauG:", tauG)
#     print("M.dot(vdot):", M.dot(vdot))
    
    #tauF = sumi(J.T*F), use object query and scenegraph inspector to calc J
    tauF = np.zeros((nv_o))
    #get query object and scenegraph inspector
    ad_query_object = ad_plant.get_geometry_query_input_port().Eval(ad_plant_context_list[context_index])
    ad_scene_graph_inspector = ad_query_object.inspector()
    
    #f is A exert to B express in W
    for n in range(N_f):
        signed_distance_pair = ad_query_object.ComputeSignedDistancePairClosestPoints(
                                    collision_candidate_pair_list[n][0], collision_candidate_pair_list[n][1])
        
        frame_A_id = ad_scene_graph_inspector.GetFrameId(signed_distance_pair.id_A)
        frame_B_id = ad_scene_graph_inspector.GetFrameId(signed_distance_pair.id_B)
        frameA = ad_plant.GetBodyFromFrameId(frame_A_id).body_frame()
        frameB = ad_plant.GetBodyFromFrameId(frame_B_id).body_frame()
        
        X_AGa = ad_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_A)
        p_GaCa = signed_distance_pair.p_ACa
        p_ACa = (X_AGa.cast[AutoDiffXd]()).multiply(p_GaCa.reshape(3, 1))
        
        X_BGb = ad_scene_graph_inspector.GetPoseInFrame(signed_distance_pair.id_B)
        p_GbCb = signed_distance_pair.p_BCb
        p_BCb = (X_BGb.cast[AutoDiffXd]()).multiply(p_GbCb.reshape(3, 1))
        
        Jv_v_WCa = ad_plant.CalcJacobianTranslationalVelocity(ad_plant_context_list[context_index], JacobianWrtVariable.kV,
                                                    frameA, p_ACa,
                                                    ad_plant.world_frame(), ad_plant.world_frame())
        Jv_v_WCb = ad_plant.CalcJacobianTranslationalVelocity(ad_plant_context_list[context_index], JacobianWrtVariable.kV,
                                                    frameB, p_BCb,
                                                    ad_plant.world_frame(), ad_plant.world_frame())
        tauF = tauF + Jv_v_WCa.T.dot(-f[n]) +  Jv_v_WCb.T.dot(f[n])
#         tauF = tauF + Jv_v_WCb.T.dot(f[n])
    #return must be 1 dimension vector, like [r1, r2, ...]
    return M.dot(vdot) + Cv - tauG - tauF


# manipulator equations for all t (implicit Euler)
for n in range(N):
    vars = np.concatenate((q_o[:, n], v_o[:, n], vdot_o[:, n], 
                           np.array(f_list[n]).flatten()))
    prog.AddConstraint(partial(manipulator_equations, context_index=n), lb=[0]*nv_o, ub=[0]*nv_o, vars=vars)

In [12]:
snopt = SnoptSolver().solver_id()
prog.SetSolverOption(snopt, 'Iterations Limits', 1e5 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Iterations Limit', 200 if running_as_notebook else 1)
prog.SetSolverOption(snopt, 'Major Feasibility Tolerance', 5e-6)
prog.SetSolverOption(snopt, 'Major Optimality Tolerance', 1e-4)
prog.SetSolverOption(snopt, 'Superbasics limit', 2000)
prog.SetSolverOption(snopt, 'Linesearch tolerance', 0.9)
#prog.SetSolverOption(snopt, 'Print file', 'snopt.out')



result = Solve(prog)
print(result.get_solver_id().name())
if not result.is_success():
#     print(result.GetInfeasibleConstraintNames(prog))
    print("solve failed!")
else:
    print("solve success")
    q_o_sol = result.GetSolution(q_o)
    print("q_o_sol:", q_o_sol)

SNOPT/fortran
solve success
q_o_sol: [[ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00

In [13]:
v_o_sol = result.GetSolution(v_o)
print("v_o_sol:", v_o_sol)

v_o_sol: [[ 0.00000000e+00  0.00000000e+00  6.93884495e-10  6.93884495e-10
   6.93884495e-10  6.93884495e-10  6.93884495e-10  6.93884495e-10
   6.93884495e-10  6.93884495e-10  6.93884495e-10  6.93884495e-10
   6.93884495e-10  6.93884495e-10  6.93884495e-10  6.93884495e-10
   6.93884495e-10  6.93884495e-10  6.93884495e-10  6.93884495e-10
   6.93884495e-10  6.93884495e-10  6.93884495e-10  6.93884495e-10
   6.93884495e-10  6.93884495e-10  6.93884495e-10  6.93884495e-10
   4.05769190e-09 -3.22768377e-09 -3.21235566e-08 -4.93268072e-08
  -5.70698641e-08 -5.75919042e-08 -5.31371536e-08 -4.59531895e-08
  -3.82892412e-08 -2.28913899e-08 -1.27016048e-08 -4.67629989e-10
   1.15619625e-08  2.43697920e-08  3.99095446e-08  5.81717425e-08
   8.09693157e-08  1.04646961e-07  1.36431235e-07  1.50730157e-07
   1.50947766e-07  1.51236736e-07  1.51597237e-07  1.12699661e-07
   6.24557972e-08  6.31444036e-09 -3.16181315e-08 -6.83174386e-08
  -1.01519125e-07 -1.29117240e-07 -1.48854643e-07 -1.58480230e-07
 

In [14]:
vdot_o_sol = result.GetSolution(vdot_o)
print("vdot_o_sol:", vdot_o_sol)

vdot_o_sol: [[ 0.00000000e+00  0.00000000e+00  3.46942247e-08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   6.72761480e-07 -1.45707513e-06 -5.77917456e-06 -3.44065013e-06
  -1.54861137e-06 -1.04408009e-07  8.90950101e-07  1.43679282e-06
   1.53278967e-06  3.07957025e-06  2.03795702e-06  2.44679496e-06
   2.40591851e-06  2.56156589e-06  3.10795053e-06  3.65243957e-06
   4.55951464e-06  4.73552915e-06  6.35685465e-06  2.85978446e-06
   4.35217496e-08  5.77940196e-08  7.21003102e-08 -7.77951523e-06
  -1.00487728e-05 -1.12282714e-05 -7.58651437e-06 -7.33986142e-06
  -6.64033724e-06 -5.51962311e-06 -3.94748056e-06 -1.92511728e-0

In [15]:
collision_point_contact_force_sol = np.zeros((N_f, 3, N))
f_list_sol = []
for n in range(N):
    f_sol = []
    for k in range(N_f):
        collision_point_contact_force_sol[k][:, n] = result.GetSolution(f_list[n][k]).reshape(3,)
        f_sol.append(result.GetSolution(f_list[n][k]))
    f_list_sol.append(f_sol)

res_f = np.zeros((3, N))
for k in range(N_f):
    res_f = res_f + collision_point_contact_force_sol[k]
print("collision_point_contact_force:", res_f)


collision_point_contact_force: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.     

In [16]:
#1-18, 5-22, 6-24, 7-20
print("collision_point0_contact_force:", collision_point_contact_force_sol[1] + 
      collision_point_contact_force_sol[5] +
      collision_point_contact_force_sol[6] +
      collision_point_contact_force_sol[7]
     )

collision_point0_contact_force: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.    

In [26]:
#1-18, 5-22, 6-24, 7-20
print("collision_candidate_pair_list:", collision_candidate_pair_list[0]
     )
print("collision_point0_contact_force:", collision_point_contact_force_sol[7]
     )

collision_candidate_pair_list: (<GeometryId value=4>, <GeometryId value=10>)
collision_point0_contact_force: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.   

In [27]:
collision_point_alpha_beta_sol = np.zeros((N_f, 2, N))
alpha_beta_list_sol = []
for n in range(N):
    alpha_beta_sol = []
    for k in range(N_f):
        collision_point_alpha_beta_sol[k][:, n] = result.GetSolution(alpha_beta_list[n][k]).reshape(2,)
        alpha_beta_sol.append(result.GetSolution(alpha_beta_list[n][k]))
    alpha_beta_list_sol.append(alpha_beta_sol)

for k in range(N_f):
    print("collision_point_alpha_beta_sol", k, ": ", collision_point_alpha_beta_sol[k])


collision_point_alpha_beta_sol 0 :  [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.

In [18]:
for n in range(N):
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], vdot_o_sol[:, n], 
                           np.array(f_list_sol[n]).flatten()))
    print("manipulator_equations:", autoDiffToValueMatrix(manipulator_equations(vars=vars, context_index=n)))

manipulator_equations: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
manipulator_equations: [[8.67361738e-18]
 [5.20417043e-18]
 [2.78880857e-13]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]
manipulator_equations: [[-4.54009660e-18]
 [-7.21916017e-16]
 [-2.78931610e-13]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
manipulator_equations: [[ 2.23328419e-17]
 [ 1.57369330e-18]
 [-2.78905569e-13]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.91069383e-13]]
manipulator_equations: [[-5.15779016e-18]
 [ 9.48038146e-18]
 [ 2.78858536e-13]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.11022302e-16]]
manipulator_equations: [[-3.75036831e-18]
 [ 2.03579165e-14]
 [-1.68875767e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-8.19344592e-14]]
manipulator_equations: [[ 5.90737060e-18]
 [-1.82775128e-18]
 [-2.09505647e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 2.22044605e-16]]
manipulator_equations: [[-5.20790997e-17]
 [ 1.20097245e-13]
 [-2.11776145e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]


In [19]:
for n in range(N):
#     print()
    #set vars be 1 dimension vector, like [r1, r2, ...]
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], 
                           np.array(alpha_beta_list_sol[n]).flatten(), 
                        np.array(f_list_sol[n]).flatten()))
    phiq_and_force_constrain_res = autoDiffToValueMatrix(phiq_and_force_constrain(vars=vars, context_index=n))
    print("phiq_and_force_constrain:", np.array([phiq_and_force_constrain_res[(6*k+3):(6*k+5), 0] for k in range(N_f)]))

phiq_and_force_constrain: [[0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.87557098e-18]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.87557098e-18]
 [0.00000000e+00 2.87557098e-18]
 [0.00000000e+00 2.87557098e-18]]
phiq_and_force_constrain: [[ 0.00000000e+00 -1.97064587e-15]
 [ 0.00000000e+00  1.01845886e-15]
 [ 0.00000000e+00 -1.97064587e-15]
 [ 0.00000000e+00 -1.97064587e-15]
 [ 0.00000000e+00 -1.97064587e-15]
 [ 0.00000000e+00 -1.93067947e-15]
 [ 0.00000000e+00 -1.93067947e-15]
 [ 0.00000000e+00 -1.93067947e-15]]
phiq_and_force_constrain: [[ 0.00000000e+00 -1.38777878e-17]
 [ 0.00000000e+00 -7.80625564e-18]
 [ 0.00000000e+00  1.38777878e-17]
 [ 0.00000000e+00  1.38777878e-17]
 [ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -6.12574227e-18]
 [ 0.00000000e+00  5.31259065e-18]
 [ 0.00000000e+00 -1.01915004e-17]]
phiq_and_force_constrain: [[ 0.00000000e+00  1.38777878e-17]
 [ 0.00000000e+00  5.42101086e-18]

phiq_and_force_constrain: [[ 0.00000000e+00 -4.02455846e-15]
 [ 0.00000000e+00  3.95516953e-15]
 [ 0.00000000e+00 -3.96904731e-15]
 [ 0.00000000e+00 -3.96904731e-15]
 [ 0.00000000e+00 -4.02455846e-15]
 [ 0.00000000e+00  4.00721123e-15]
 [ 0.00000000e+00  4.00374178e-15]
 [ 0.00000000e+00  3.95516953e-15]]
phiq_and_force_constrain: [[ 0.00000000e+00 -4.02455846e-15]
 [ 0.00000000e+00  3.95170008e-15]
 [ 0.00000000e+00 -3.96904731e-15]
 [ 0.00000000e+00 -3.95516953e-15]
 [ 0.00000000e+00 -4.01068068e-15]
 [ 0.00000000e+00  4.00721123e-15]
 [ 0.00000000e+00  4.03149736e-15]
 [ 0.00000000e+00  3.97598621e-15]]
phiq_and_force_constrain: [[ 0.00000000e+00 -4.02455846e-15]
 [ 0.00000000e+00  3.92394450e-15]
 [ 0.00000000e+00 -3.96904731e-15]
 [ 0.00000000e+00 -3.92741395e-15]
 [ 0.00000000e+00 -3.99680289e-15]
 [ 0.00000000e+00  4.00027234e-15]
 [ 0.00000000e+00  4.02108902e-15]
 [ 0.00000000e+00  3.95863897e-15]]
phiq_and_force_constrain: [[ 0.00000000e+00 -4.02455846e-15]
 [ 0.00000000e+00 

In [20]:
with open(tmpfolder + 'test_only_box_passive_scenegraph/sol.pkl', 'wb') as file:
    pickle.dump( [q_o_sol, v_o_sol, vdot_o_sol, f_list_sol, alpha_beta_list_sol], file )

In [26]:
with open(tmpfolder +  'test_only_box_passive_scenegraph/sol.pkl', 'rb' ) as file:
          q_o_ini, v_o_ini, vdot_o_ini, f_list_ini, alpha_beta_list_ini = pickle.load( file )
print("q_o_sol:", ql)
print("v_o_sol:", vl)
print("vdot_o_sol:", vdotl)
print("f_list_sol:", fl)
print("alpha_beta_list_sol:", alpha_betal)

q_o_sol: [[1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.        1.        1.        1.        1.        1.
  1.        1.       ]
 [0.        0.        0.        0.        0.        0.    

In [40]:
print(q0_o)
print(v0_o)

[1.     0.     0.     0.     0.     0.     0.0334]
[0. 0. 0. 0. 0. 0.]


In [17]:
def manipulator_equations_val(vars):
    nf = 3
    # split input vector in subvariables
    # configuration, velocity, acceleration, collision_point force
    split_at = [nq_o, nq_o + nv_o, nq_o + 2 * nv_o, nq_o + 2 * nv_o + nf, nq_o + 2 * nv_o + 2 * nf, nq_o + 2 * nv_o + 3 * nf]
    q, v, vdot, f1, f2, f3, f4 = np.split(vars, split_at)
    f = [f1, f2, f3, f4]

    plant_context = plant.CreateDefaultContext()
    plant.SetPositions(plant_context, q)
    plant.SetVelocities(plant_context, v)
    
    # matrices for the manipulator equations
    M = plant.CalcMassMatrixViaInverseDynamics(plant_context)
    Cv = plant.CalcBiasTerm(plant_context)
    tauG = plant.CalcGravityGeneralizedForces(plant_context)
    print("tauG:\n" , tauG)
    print("g:\n" , plant.gravity_field().gravity_vector())
    # Jacobian of the collision_point
    # get reference frames for collision_point
    collision_point_frame = [
    plant.GetFrameByName('point_collision5'),
    plant.GetFrameByName('point_collision6'),
    plant.GetFrameByName('point_collision7'),
    plant.GetFrameByName('point_collision8')]

    # compute Jacobian matrix
    J = [plant.CalcJacobianTranslationalVelocity(
        plant_context,
        JacobianWrtVariable.kV,
        collision_point_frame[collision_point],
        [0., 0., 0.],
        plant.world_frame(),
        plant.world_frame())
        for collision_point in range(4)]
    

    tauF = np.zeros((nv_o))
    for collision_point in range(4):
        tauF = tauF + J[collision_point].T.dot(f[collision_point])
        print("J", collision_point, ": \n" , J[collision_point])
        
    print("#######################")
    # return violation of the manipulator equations
    return M.dot(vdot) + Cv - tauG - tauF

In [18]:
for n in range(1):
    vars = np.concatenate((q_o_sol[:, n], v_o_sol[:, n], vdot_o_sol[:, n], 
                           collision_point_contact_force_sol[0][:, n], collision_point_contact_force_sol[1][:, n],
                           collision_point_contact_force_sol[2][:, n], collision_point_contact_force_sol[3][:, n]))
    print("manipulator_equations:", manipulator_equations_val(vars))

tauG:
 [-0.    -0.    -0.    -0.    -0.    -0.981]
g:
 [ 0.    0.   -9.81]
J 0 : 
 [[ 0.     -0.0334 -0.1042  1.      0.      0.    ]
 [ 0.0334  0.      0.0795  0.      1.      0.    ]
 [ 0.1042 -0.0795  0.      0.      0.      1.    ]]
J 1 : 
 [[ 0.     -0.0334  0.1042  1.      0.      0.    ]
 [ 0.0334  0.      0.0795  0.      1.      0.    ]
 [-0.1042 -0.0795  0.      0.      0.      1.    ]]
J 2 : 
 [[ 0.     -0.0334 -0.1042  1.      0.      0.    ]
 [ 0.0334  0.     -0.0795  0.      1.      0.    ]
 [ 0.1042  0.0795  0.      0.      0.      1.    ]]
J 3 : 
 [[ 0.     -0.0334  0.1042  1.      0.      0.    ]
 [ 0.0334  0.     -0.0795  0.      1.      0.    ]
 [-0.1042  0.0795  0.      0.      0.      1.    ]]
#######################
manipulator_equations: [0.0000000e+00 6.9388939e-18 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00]


In [19]:
p_WF = plant.CalcPointsPositions(plant_context, plant.GetFrameByName('point_collision5'), [0,0,0], plant.world_frame())
print(p_WF)
X_WF = plant.CalcRelativeTransform(plant_context, plant.GetFrameByName('point_collision5'), plant.world_frame())
print(X_WF)

[[0.]
 [0.]
 [0.]]
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


In [19]:
print(gravity)

[ 0.    0.   -9.81]
